# STEP36 高階微分以外の用途

## これまでの流れ

- STEP27:テイラー展開の微分
- STEP28:ローゼンブロック関数を勾配降下法を理解する
- STEP29:ニュートン法を用いた最適化, 勾配法は遅いのでニュートン法による収束高速化を目指す
- STEP30:Back propagation の実装を見直して2階微分, 3階微分など高階微分に対応できるようにする(準備)
- STEP31:Back propagation の実装を見直して2階微分, 3階微分など高階微分に対応できるようにする(理論)
- STEP32:Back propagation の実装を見直して2階微分, 3階微分など高階微分に対応できるようにする(実装)
- STEP33:ニュートン法は2階微分を使うので、STEP30〜STEP32までの成果を使って自動に最適化計算を行ってみる
- STEP34:新しい関数としてsin関数/cos関数を実装する
- STEP35:新しい関数としてtanhを追加する
- STEP36:double backpropの活用例を理解する

## 事前準備

In [2]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import Variable

## 36.1 double backpropの用途

### テスト関数の設定

次の２つの式が与えられたとき, z=2.0における $\frac{\partial z}{\partial x}$ を求めよ

$$
\begin{align}
& y=x^2 \\
& z = \left( \frac{\partial y}{\partial x} \right)^3 + y\\
\end{align}
$$

### 解析的に解いて正しい解を求めておく...

上記を式変形し、zとxの関係に整理すると

$$
\begin{align}
& z = \left( \frac{\partial y}{\partial x} \right)^3 + y = 8x^3 + x^2 \\
& \frac{\partial z}{\partial x} = 24x^2 + 2x
\end{align}
$$

x = 2.0 ならば 

$$
\begin{align}
& \frac{\partial z}{\partial x} = 24 \times 2^2 + 2 \times 2 = 96 + 4 = 100
\end{align}
$$

### 上記の関数をDeZeroで解いてみる

#### STEP1) $y = x^2$ と、その微分を計算する

- `y.backward(create_graph=True)` の `create_graph=True` を指定することがポイント
    - これにより逆伝搬のときに、新しい計算グラフが作成される

$ y=x^2 $ と $\frac{dy}{dx}$を予め計算

In [3]:
x = Variable(np.array(2.0))
y = x ** 2
y.backward(create_graph=True)
gx = x.grad
x.cleargrad()

#### STEP2) 上記で求めた微分値を使って、その微分を計算する

$z = {g_x}^3 + y$ を計算

In [5]:
z = gx ** 3 + y
z.backward()
print(x.grad)

variable(100.0)


## 36.2 Deep Learningの研究での使用例

### WGAN-GPの論文における double backprop 活用例

以下のようなLの最適化で、**double backprop** を利用する例あり

$$
\begin{align}
L={\underset{\tilde{x}\sim\mathbb{P}_g}{\mathbb{E}}}\left[D\left(\tilde{\mathbf{x}}\right)\right]
-{\underset{\tilde{x}\sim\mathbb{P}_r}{\mathbb{E}}}\left[D\left(\mathbf{x}\right)\right]+\lambda{\underset{\tilde{x}\sim\mathbb{P}_\hat{\omega}}{\mathbb{E}}}\left[\left(\parallel \triangledown_\hat{x} D\left(\tilde{\mathbf{x}}\right)\parallel_2-1\right)^2\right]
\end{align}
$$

- $\triangledown_\hat{x} D\left(\tilde{\mathbf{x}}\right)$ は勾配であり、つまり微分である
- この勾配は逆伝搬で計算できる
- 計算した勾配を使って関数Lを計算する
- 関数Lを最適化するために2回目の逆伝搬を行う

### その他の論文におけるdouble backprop活用例

- MAML
- TRPO
    - ヘッセ行列とベクトルの積を求める際に、double backpropを使う

## コラム: ニュートン法とdouble backpropの補足

以下を説明する

- 入力がベクトルの場合のニュートン法
- ニュートン法に変わる別の手法
- double backpropの実用的な用途

### 多変数関数のニュートン法 (結局のところニュートン法は最適化には使われない)

以下のような、多次元の入力変数 $\mathbf x$ をもつ関数 $y$ を考える

$$
\begin{align}
& y=f(\mathbf x)\\
& \mathbf x = \left(x_1,x_2,x_3,...,x_n\right)
\end{align}
$$

このとき多次元のニュートン法は以下のように表される。

$$
\begin{align}
& \mathbf x \leftarrow \mathbf x - \left[\triangledown^2f\left(\mathbf x\right)\right]^{-1}\triangledown f\left(\mathbf x\right)
\end{align}
$$

ここで $\left[\triangledown^2f\left(\mathbf x\right)\right]^{-1}$ は $\triangledown^2f\left(\mathbf x\right)$ の逆行列であり
$\triangledown f\left(\mathbf x\right)$, $\triangledown^2f\left(\mathbf x\right)$ はそれぞれ以下のように表現できる


$$
\begin{align}
& \triangledown f\left(\mathbf x\right)=\left(\begin{array}{c}\frac{\partial f}{\partial x_1}\\ \frac{\partial f}{\partial x_2}\\:\\\frac{\partial f}{\partial x_n}\end{array}\right)
\end{align}
$$

$$
\begin{align}
& \triangledown^2 f\left(\mathbf x\right)=\left(\begin{array}{c}\frac{\partial^2 f}{\partial {x_1}^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2} & ...& \frac{\partial^2 f}{\partial x_1 \partial x_n}\\ \frac{\partial^2 f}{\partial^2 x_2 \partial x_1} & \frac{\partial^2 f}{\partial{x_2}^2} & ... & \frac{\partial^2 f}{\partial x_2 \partial x_n}\\: & : &...&:\\\frac{\partial^2 f}{\partial x_n \partial x_1} & \frac{\partial^2 f}{\partial x_n \partial x_2} & ... & \frac{\partial^2 f}{\partial {x_n}^2}
\end{array}\right)
\end{align}
$$

### ニュートン法の問題点

ヘッセ行列の逆行列を計算するのが大変。パラメータの数が100万個を超えると1M x 1M のヘッセ行列が必要になる。

- メモリ不足の問題
- 計算時間の問題

ニュートン法に変わるアプローチ

- 准ニュートン法: ヘッセ行列の逆行列を近似して使用する
    - L-BFGS: 准ニュートン法で特に有名な手法 
        -　勾配だけからヘッセ行列を近似
        - PyTouchで実装されている
- 現状の主流は以下であり准ニュートン法も使われる機会はあまりない
    - SGD
    - Momentum
    - Adam

### double backpropの用途: ヘッセ行列と行列の積

ヘッセ行列を求める計算コストは高いが、ヘッセ行列とベクトルの積の結果だけが必要であれば、double backpropを使い効率よく求めることができる

$$
\begin{align}
\triangledown^2f\left(\mathbf x\right)\mathbf v = \triangledown \left(  \mathbf v^T f\left( \mathbf x\right) \right)
\end{align}
$$

ここで、

\begin{align}
\triangledown^2 f\left(\mathbf x\right)\mathbf v 
& =\left(\begin{array}{c}\frac{\partial^2 f}{\partial {x_1}^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2}\\ \frac{\partial^2 f}{\partial x_2 \partial x_1} & \frac{\partial^2 f}{\partial{x_2}^2}
\end{array}\right)\left(\begin{array}{c}v_1\\ v_2\end{array}\right) \\
& =\left(\begin{array}{c}\frac{\partial^2 f}{\partial {x_1}^2}v_1 + \frac{\partial^2 f}{\partial x_1 \partial x_2}v_2\\ \frac{\partial^2 f}{\partial x_2 \partial x_1}v_1 + \frac{\partial^2 f}{\partial{x_2}^2}v_2
\end{array}\right)
\end{align}

$$
\begin{align}
\triangledown \left(  \mathbf v^T f\left( \mathbf x\right) \right)
& =\triangledown \left(v_1, v_2\right) 
\left(\begin{array}
{c}\frac{\partial f}{\partial {x_1}} \\ \ \frac{\partial f}{\partial{x_2}}
\end{array}\right) \\
& =\triangledown \left(
v_1\frac{\partial f}{\partial {x_1}} + v_2\frac{\partial f}{\partial{x_2}}
\right) \\
& =\left(\begin{array}{c}\frac{\partial^2 f}{\partial {x_1}^2}v_1 + \frac{\partial^2 f}{\partial x_1 \partial x_2}v_2\\ \frac{\partial^2 f}{\partial x_2 \partial x_1}v_1 + \frac{\partial^2 f}{\partial{x_2}^2}v_2
\end{array}\right)
\end{align}
$$

### 以下のコラムのコードは現状では動かない

```python
import numpy as np
from dezero import Variable
import dezero.functions as F
import dezero.utils

x = Variable(np.array([1.0, 2.0]))
v = Variable(np.array([4.0, 5.0]))

def f(x):
    t = x ** 2
    y = F.sum(t)
    return y

y = f(x)
y.backward(create_graph=True)

gx = x.grad
x.cleargrad()

z = F.matmul(v, gx)
z.backward()
print(x.grad)
```